# **Classical Methods:** Introduction to Molecular Dynamics with LAMMPS and ASE

As you've probably heard in the seminars until now, **Molecular Dynamics (MD)** is a computational simulation method used to model the physical movements of particles (typically atoms and molecules, but not only) over time. By numerically solving (technicaly, *integrating*) Newton's equations of motion, MD allows us to investigate the dynamical evolution of a system based on the forces acting between particles. These forces can be derived from *ab initio* methods or obtained from empirical models. *(Side note: machine-learned interatomic potentials, or MLIPs, can also be considered a type of empirical potential when trained on reference data.)*

This first half of the notebook was designed to give you a taste of how we can use **ASE (Atomic Simulation Environment)** together with **LAMMPS**, a powerful classical MD software, to set up and run a basic MD simulation.


## First, a "simple" MD...
As a first test case, we'll model **aluminium (Al)**, a widely used metal with a simple FCC (face-centered cubic) crystal structure that is easy to [build using ASE](https://wiki.fysik.dtu.dk/ase/ase/build/build.html). We'll perform a **temperature ramp simulation** of bulk aluminium using **periodic boundary conditions** to mimic an infinite crystal. Starting at a relatively low temperature (100 K), we will gradually increase the system temperature to 2000 K over 20,000 simulation steps. This controlled heating will help illustrate how thermal energy influences atomic motion and structure.

In this exercise, we'll use a many-body empirical potential known as the **Embedded Atom Method (EAM)** to describe atomic interactions. The specific potential file used can be retrieved from [this link](https://www.ctcms.nist.gov/potentials/entry/2009--Zhakhovskii-V-V-Inogamov-N-A-Petrov-Y-V-et-al--Al/). EAM is an oldschool potential for modeling metallic systems, and even today it is widely used for its simplicity and well available parametrizations. It considers not only pairwise interactions between atoms but also the **embedding energy** associated with placing an atom into the local electron density generated by its neighbors. In this methodology, the total energy $E$ of a system of atoms is given by:

$
E = \sum_i F_i(\bar{\rho}_i) + \frac{1}{2} \sum_{i \ne j} \phi_{ij}(r_{ij})
$

Where:
- $F_i(\rho_i)$ is the **embedding energy** for atom $i$, a function of the local electron density $\rho_i$.
- $\bar{\rho}_i = \sum_{j \ne i} \rho_j(r_{i})$ is the local electron density at atom $i$ position, resulting from all neighboring atoms $j$.
- $\phi_{ij}(r_{ij})$ is a pairwise potential interaction between atoms $i$ and $j$.

If you are interested in understanding it a little more, try to check [this reference](https://doi.org/10.1103/PhysRevB.29.6443) or [the EAM entry at LAMMPS manual](https://docs.lammps.org/pair_eam.html).


Let's dive into the code and start building our MD script! Try to follow the comments and guide yourself through the code below. Once you're comfortable, get your hands dirty and try modifying the script. We've also included some challenges further down to help you test your understanding and progress.

In [ ]:
# This cell aims to install LAMMPS and ASE. You can ignore the verbose outputs that come from it if you dont have any major error running it.
!apt-get -qq update
!pip --quiet install -U ase lammps==2024.8.29.3.0
# We are fixing this version cause the last release of lammps for pip has some issues with colab.
!pip --quiet install -U nglview py3Dmol
# Some people can have errors like:
#    `google-colab x.x.x requires notebook==x.x.x, but you have notebook x.x.x which is incompatible.`
# This should not interfeers with any of the practices bellow. Be free to ignore it.

In [ ]:
# If your enviropment is correctly configured, this command should return the full path for LAMMPS executable
!which lmp_serial || which lmp || find /usr -name "lmp*" 2>/dev/null

In [ ]:
# We strongly recomend you to go on the NIST database andf check the potentials by yourself: https://www.ctcms.nist.gov/potentials/
# the one we will be using here can be automaticaly downloaded with this cell.
# Check the folder icon in the left to see if the file is realy downloaded.

!wget https://www.ctcms.nist.gov/potentials/Download/2009--Zhakhovskii-V-V-Inogamov-N-A-Petrov-Y-V-et-al--Al/2/Al-2009.eam.alloy

In [ ]:
################################################################
####      START THIS CELL BEFORE READING IT!
####
####    It can take around 5 min to run.
####    Yeah, this is a simple example... MD can be expencive.
################################################################

# Importing ASE functionalities
from ase import units
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib
from ase.io import write
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary, ZeroRotation
from ase.md.nvtberendsen import NVTBerendsen
#Other utility libraries we will need:
import os
import numpy as np
import pickle
import time

# The goal of the following code is:
#
# - Build an aluminum supercell using ASE.
# - Assign initial velocities using a Maxwell-Boltzmann distribution.
# - Interface ASE with LAMMPS using LAMMPSlib.
# - Run a temperature-controlled MD simulation using the NVT Berendsen thermostat.
# - Save and inspect the resulting atomic trajectories.

#Lets store  the cpu time before starting the calculations...
t0 = time.time()

# Cheking for the EAM parameter file.
# Make sure the file is present, the link to download it was given in the introduction.
potential_file = "Al-2009.eam.alloy"
assert os.path.isfile(potential_file), f"Missing {potential_file}!"

# Ramp parameters
T_start = 100    # K
T_end = 2000     # K
n_steps = 20000
timestep_fs = 1.5
vscale = (2.697/2.304)
# We used vscale here to adjust the standard Al unit cell and have a density more kin to the liquid.
# Can you understand how it was done? How would you do it differently?

# Build supercell, knowing the lattice parameter (4.0495 Angstrom) and considering vscale.
atoms = bulk('Al', cubic=True, a=4.0495 * vscale**(1 / 3)).repeat((5,5,5))

# Setup LAMMPSlib calculator
# A calculator is an object that ASE uses to comput physical quantities for your atomistic systems.
# Its sintaxe can change significantly from one code to another, so check the documentation here:
#      https://wiki.fysik.dtu.dk/ase/ase/calculators/lammpslib.html#ase.calculators.lammpslib.LAMMPSlib
# Otherwise, lets for now stay as simple as possible:
lmp = LAMMPSlib(
    lmpcmds=[
        "pair_style eam/alloy",
        f"pair_coeff * * {potential_file} Al"
    ],
    atom_types={'Al': 1},
    log_file=None,
    keep_alive=True
)
# Here we atribute this calculator to our supercell
atoms.calc = lmp

# The folloing lines initialize the velocities using the Maxwell Boltzmann Distributionand
# and make sure that the final system has 'zero motion' (total linear and angular velocities are zero)
print(f"- Setting up NVT temperature ramp from {T_start} K to {T_end} K over {n_steps} steps...")
MaxwellBoltzmannDistribution(atoms, temperature_K=T_start)
ZeroRotation(atoms)
Stationary(atoms)

# Create NVT Berendsen dynamics
# What Berendsen means here? You can take it as a homework, but its understanding
# touches the concepts of thermostat in MD and will make a little more clear to you
# how temperature is controled in this model.
# https://wiki.fysik.dtu.dk/ase/ase/md.html#module-ase.md.nvtberendsen
dyn = NVTBerendsen(
    atoms,
    timestep=timestep_fs * units.fs,
    temperature_K=T_start,
    taut=10 * units.fs
)
temperatures = np.linspace(T_start, T_end, n_steps)

# To have a complete log of the MD, we are also deffining a Logger, a subroutine that
# register the state of the calculation at each time step.
traj_data = []
def log_step():
    step_idx = dyn.nsteps
    T_current = temperatures[step_idx] if step_idx < n_steps else T_end
    dyn.set_temperature(T_current)

    epot = atoms.get_potential_energy()
    temp = atoms.get_temperature()
    forces = atoms.get_forces()
    traj_data.append({
        "step": step_idx,
        "atoms": atoms.copy(),
        "energy": epot,
        "temperature": temp,
        "forces": forces
    })
dyn.attach(log_step, interval=1)

# Running the MD itself!

print("- Running MD...")
dyn.run(n_steps)

# Now, lets store our results in files:
xyz_filename = f"traj_ramp_{T_start}K_to_{T_end}K.xyz"
traj_cut=[traj_data[i]["atoms"] for i in list(range(0,len(traj_data)+1,100))] # Printing at each 100 steps
write(xyz_filename, [frame for frame in traj_cut])
print(f"- Trajectory saved to {xyz_filename}")

# Lets also save all data in pickle, just in case we need it later
with open("al_md_ramp_lammpslib.pkl", "wb") as f:
    pickle.dump(traj_data, f)

#Finnaly, lets see how long it took to do this job! Usualy a Colab vitrtual machine takes ~250s
print("\n- Temperature ramp simulation complete.")
tf = time.time()
print(f"Total time: {tf - t0:.2f} seconds")

In [ ]:
# This is how you recover the properties we saved in traj_data, in case you need for a future practice! (10 and 20 are examples)
index_step=2000
index_atom=0
print("Temperature at this step: ",traj_data[index_step]["temperature"])
print("Energy at this step: ",traj_data[index_step]["energy"])
print("Positions of the piked atom:  ",traj_data[index_step]["atoms"].get_positions()[index_atom])
print("Atomic symbols of the piked atom:  ",traj_data[index_step]["atoms"].get_chemical_symbols()[index_atom])
print("Atomic masses of the piked atom:  ",traj_data[index_step]["atoms"].get_masses()[index_atom])
print("Forces at this step on the piked atom:  ",traj_data[index_step]["forces"][index_atom])


In [ ]:
# This cell includes a pair of plots that show termodynamical properties of the system evolving.
# Can you explain what we see in those figures?

import matplotlib.pyplot as plt

# Extract temperature and energy from trajectory
temps = np.array([frame["temperature"] for frame in traj_data])
energies = np.array([frame["energy"] for frame in traj_data])

# Block averaging: define block size
block_size = 800  # You can adjust this
step = 100
n_blocks = ((len(temps) - block_size) // step) - 1

# Arrays to store block-averaged data
block_temps = []
block_energies = []
block_varE = []
block_varT = []

for i in range(n_blocks):
    start = i * step + block_size // 2
    end = start + block_size
    t_block = temps[start:end]
    e_block = energies[start:end]

    block_temps.append(np.mean(t_block))
    block_energies.append(np.mean(e_block))
    block_varE.append(np.var(e_block))
    block_varT.append(np.var(t_block))

# Convert to numpy arrays
block_temps = np.array(block_temps)
block_energies = np.array(block_energies)
block_varE = np.array(block_varE)
block_varT = np.array(block_varT)
varE_over_varT = block_varE / block_varT

# --- Plot Energy vs Temperature ---
plt.figure(figsize=(8, 5))
plt.plot(block_temps, block_energies, '-o', label='E(T)')
plt.xlabel("Temperature (K)")
plt.ylabel("Potential Energy (eV)")
plt.title("Energy vs Temperature")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Plot var(E)/var(T) vs Temperature ---
plt.figure(figsize=(8, 5))
plt.plot(block_temps, varE_over_varT, '-o', color='red', label='var(E)/var(T)')
plt.xlabel("Temperature (K)")
plt.ylabel("var(E) / var(T)")
plt.title("Variance Ratio vs Temperature")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#Lets quicklyh build a visualization block. You can also download traj_ramp_100K_to_2000K.xyz and use OVITO, VMD, xmakemol...
from ase.io import read
from ase.visualize import view

# Load all frames from the trajectory
traj = read("traj_ramp_100K_to_2000K.xyz", ":")

# Wrap all atoms into the unit cell
for frame in traj:
    frame.wrap()

# Visualize the entire animation
view(traj[10], viewer='x3d',)


In [ ]:
# Another alternative is to use py3Dmol to generate the visualization.
import py3Dmol
from ase.io import read
from ase import Atoms
import io

# Load trajectory
traj = read("traj_ramp_100K_to_2000K.xyz", ":")

# Wrap atoms into the unit cell for each frame
for frame in traj:
    frame.wrap()

# Convert trajectory to XYZ string format
xyz_str = ""
for atoms in traj:
    f = io.StringIO()
    atoms.write(f, format='xyz')
    xyz_str += f.getvalue()

# Visualize with py3Dmol
view = py3Dmol.view(width=600, height=400)
view.addModelsAsFrames(xyz_str, 'xyz')

# Set to 'sphere' representation for ball-style atoms
view.setStyle({'sphere': {'radius' : 1.0}})

# Animation controls
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

### What can you do?

1.   How would you adapt this code to simulate Ti? Can you find a paramaterization for it in the Interatomic Potentials Repository? What is the unit cell shape and lattice parameter? `vscale` will change?
2.   Do you think you can go even foward with the structures been used? Maybe even another potential... Try building a new cell for a systems like FeO or Cu3Au. In case you wanna run the dynamics, there maybe be available potentials for you to try in the same repository or in papers on the literature, what imply also in changing the `pair_style` and `pair_coeff` lines in our code.
3.   There is a lot of methods for the ASE `atoms` object, and you can check a full list [here](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#working-with-the-array-methods-of-atoms-objects). Can you incorporate then in the logger and get a new plot? A good one for instance would be to plot the evolution of the six independent components of the symmetric stress tensor. Thet can be obtained in the traditional Voigt order (i.e., as $[xx, yy, zz, yz, xz, xy]$) from `atoms.get_stress()`. Be creative with your plot and try to see if any patern appears in the heating.



In [ ]:
# [Your time to shine!]



---



---



---



## Lets make a workflow
In computational materials science and chemistry, a **workflow** is a very loosely used term to refer to a structured sequence of tasks or simulations that are logically chained together to achieve a broader objective. Rather than running a single calculation in isolation, workflows help us build more complex simulations by automating steps such as system setup, simulation, data extraction, and post-processing.

Workflows will be essencial when you want to study how a system evolves across a range of conditions (e.g., temperature, pressure, composition), you need to reuse results from one step as the starting point for the next, you want to ensure consistency and reproducibility across simulations... If some few keywords can remain in your mind after this very broad definition, you should keep the information that workflows aim to be **repeatable** and **as automated as possible**.


In this section, we will build a **simple MD workflow** that performs a sequence of NVT simulations of aluminium at different temperatures. We will now perform a series of **NVT simulations** of bulk aluminium at constant temperatures ranging from **750 K to 1250 K**, in steps of 100 K. The goal is to observe how thermal energy affects atomic dynamics and equilibrium structure over those temperatures, but you can also compare the results to the ones we obteined in the ramp before.

Rather than repeating the setup code for each simulation manually, lets design our script so that:
- The atomic structure is initialized **once** at the beginning.
- Each simulation runs at a fixed temperature for, lets say, 2000 steps.
- The **final structure** from one simulation becomes the **starting point** for the next, preserving thermal history.
- Simulation data (e.g. temperature, potential energy) is collected for analysis after all runs are complete.

This is a common workflow pattern in MD studies where thermal annealing or sequential temperature sweeps are required. We are providing you with a 'skeleton' code for this task, you should try to fill it with the knowledge of the previous code.


In [ ]:
from ase import Atoms
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib
from ase import units
from ase.io import write
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary, ZeroRotation
from ase.md.nptberendsen import NPTBerendsen, NVTBerendsen
import time
import os
import numpy as np
import pickle

t0=time.time()

potential_file = "Al-2009.eam.alloy"
assert os.path.isfile(potential_file), f"Missing {potential_file}!"

temperatures = range(750, 1251, 100)
n_steps = 2000
timestep_fs = 1.5
vscale=(2.697/2.304)


# atoms = bulk(...)
# (Remember to also initialize the velocities!)

all_data = []
for T in temperatures:
    print(f"\n- Running MD at {T} K")

    # Define the calculator
    # lmp = ???
    # Extra question: This needs to be done inside the loop?
    atoms.calc = lmp

    dyn = NVTBerendsen(
        # ???
    )

    traj_data = []
    # Note: The logger dont include the temperature changing, but still register
    # the instantaneous temperature from the MD itself.
    def log_step():
        epot = atoms.get_potential_energy()
        temp = atoms.get_temperature()
        traj_data.append({
            "atoms": atoms.copy(),
            "energy": epot,
            "temperature": temp
        })
    dyn.attach(log_step, interval=10)

    print("   - Starting NVT run...")
    # ?????????????????

    all_data.append({
        "T_K": T,
        "trajectory": traj_data
    })

    xyz_filename = f"traj_T{T}K.xyz"
    write(xyz_filename, [frame["atoms"] for frame in traj_data])
    print(f"   - Trajectory saved to {xyz_filename}")

# Save all data
with open("al_md_lammpslib.pkl", "wb") as f:
    pickle.dump(all_data, f)

print("\n\n- All simulations complete. Results saved to 'al_md_lammpslib.pkl'")
tf=time.time()
print(f"Total time: {tf-t0} seconds")


In [ ]:
# Here, some plots as ideas for you to play with...

import matplotlib.pyplot as plt

# Helper: Get time array (in fs)
def get_time_array(n_frames, interval, timestep_fs):
    return np.arange(n_frames) * interval * timestep_fs

# Plot Energy vs Time
plt.figure(figsize=(10, 6))
for run in all_data:
    T = run["T_K"]
    energies = [f["energy"] for f in run["trajectory"]]
    time = get_time_array(len(energies), 10, timestep_fs)
    plt.plot(time, energies, label=f"{T} K")

plt.title("Potential Energy vs Time")
plt.xlabel("Time (fs)")
plt.ylabel("Energy (eV)")
plt.legend()
plt.grid(True)
plt.show()

# Plot Energy vs Instantaneous Temperature
plt.figure(figsize=(10, 6))
for run in all_data:
    T = run["T_K"]
    energies = [f["energy"] for f in run["trajectory"]]
    temps = [f["temperature"] for f in run["trajectory"]]
    plt.scatter(temps, energies, s=10, alpha=0.5, label=f"{T} K")

plt.title("Energy vs Instantaneous Temperature")
plt.xlabel("Temperature (K)")
plt.ylabel("Energy (eV)")
plt.legend()
plt.grid(True)
plt.show()

# Plot Temperature vs Time (thermalization behavior)
plt.figure(figsize=(10, 6))
for run in all_data:
    T = run["T_K"]
    temps = [f["temperature"] for f in run["trajectory"]]
    time = get_time_array(len(temps), 10, timestep_fs)
    plt.plot(time, temps, label=f"{T} K")

plt.title("Temperature vs Time")
plt.xlabel("Time (fs)")
plt.ylabel("Instantaneous Temperature (K)")
plt.legend()
plt.grid(True)
plt.show()

Some other exercices for you to think about:

1.   In some of the plots, is very clear that the initial steps of the simulation are a transient regime and the system is acomodating to the thermodynamic conditions. Can you change those plots to not include this first $N$ steps? (You can estimate $N$ analysing the plots as they are right now.)
2.   From thermodynamics, we know $c_V=\frac{1}{M}\left.\frac{dQ}{dT}\right|_V$. How would you go towards ploting the $c_V$ for each one of the temperatures above?
3.   It seems that we are seen a phase transition. How would you try to determine its critical temperature? (Some soluctions to this problem can be computationaly expensive. Lets, for today, just think about the solution, no implementation necessary.)

In [ ]:
#[Here there will be dragons...]



---



---



---



# ***Ab initio* Methods:** Introduction to Single Point, Vibrational modes and Band Structure calculations with PySCF


## Basics of PYSCF for molecular systems

PySCF (Python-based Simulations of Chemistry Framework) is an open-source quantum chemistry package designed for coders, focusing in simplicity and flexibility. Written in Python with efficient C backend components, PySCF allows users to easily build, customize, and automate quantum chemistry calculation. It supports a wide range of methods, including Hartree–Fock (HF), Density Functional Theory (DFT), post-HF methods, and more. It is far from the more efficient code in the literature when we talk about computational performacy, where other codes (like FHI-aims, ORCA, VASP, Quantum Espresso, ...) should be prioritary. PySCF is aimed for development and modularity, or to run small systems.

In this section, we introduce the basic works of PySCF for molecular systems, using a series of hands-on examples to demonstrate how to define a molecule, perform electronic structure calculations, analyze results, and visualize key properties such as molecular orbitals, dipole moments, and electronic spectra.


In [ ]:
!pip --quiet install pyscf ase py3Dmol

In [ ]:
##### A very straightforward exemple with minimal information
from pyscf import gto
from pyscf import scf
import time

#Basic Molecule geometry and proprieties parsing. More features are explained in https://pyscf.org/user/gto.html
mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    basis = '6-311g')

#Exemple of a basic unrestricted Hartree--Fock calculation.
#mf.kernel returns the total energy, nothing else should be printed with mf.verbose = 0
mf = scf.UHF(mol)
mf.verbose = 0

t0=time.time()
e_RHF = mf.kernel()
tf = time.time()
print(f"Time on the Single Point calculation: {tf-t0} seconds")
print(f"Total HF energy: {e_RHF}")


In [ ]:
##### A similar exemple, now with few keywords added to demonstrate PySCF functionalities
from pyscf import dft,gto,scf

#Molecule Definition
mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    charge = 0,                  # -1 = one extra electron
    spin = 0,                    # Number of unpared electrons (same as 2S)
    output = 'output.out',       # Direct output of calculation for this molecule to a file
    basis = '6-311g'             # Basis set used on the SCF process
    )


#Same molecule, but now with a different method, restricted DFT (PBE).
mf = scf.RKS(mol).newton()        # .newton() activates second-order self-consistent field (SOSCF)
mf = scf.addons.frac_occ(mf)      # Allow fractional orbital ocupation to help convergence

mf.xc = 'PBE'                     # Functional PBE. Any functional in XCLib can be used
mf.verbose = 4                    # Defines verbosity level. 0 = no output
mf.conv_tol = 1e-10               # Energy convergence criteria, in Eh
mf.max_cycle = 100                # Maximun allowed number of SCF cycles


mf.chkfile = 'chkpoint.dat'       # Checkpoint file to restart calculations
mf.init_guess = 'vsap'            # Strategy used for initial density guess
#mf.init_guess = 'chkpoint.dat'   # Loading old ckeckpoint file


#Runs the SCF
mf.kernel()



In [ ]:
##### Lets get some other physical quantities out of our calculation:

#Analyze the given SCF object, this is what computes Mulliken population and orbital info
results = mf.analyze()

print("Molecular Properties Computed from PySCF")

# Dipole Moment - 3D vector in Debye. It represents the separation of positive and negative charges.
dipole = mf.dip_moment(unit='Debye')
dipole_magnitude = np.linalg.norm(dipole)
print(f"\nDipole Moment: {dipole_magnitude:.3f} D")
print(f"   Components   : x = {dipole[0]:.3f}, y = {dipole[1]:.3f}, z = {dipole[2]:.3f} (Debye)")

# HOMO-LUMO Gap - The gab in energy between the highest occupied AND the lowest unoccupied molecular orbital.
mo_energy = mf.mo_energy
homo_index = mol.nelectron // 2 - 1
lumo_index = homo_index + 1
homo_energy = mo_energy[homo_index]
lumo_energy = mo_energy[lumo_index]
gap_au = lumo_energy - homo_energy
gap_ev = gap_au * 27.2114  # 1 Hartree = 27.2114 eV
print(f"\nHOMO-LUMO Gap: {gap_ev:.2f} eV ({gap_au:.4f} Hartree)")
print(f"   HOMO Energy  : {homo_energy:.4f} Hartree")
print(f"   LUMO Energy  : {lumo_energy:.4f} Hartree")

# Mulliken Atomic Charges - A first approximation of electron population in the atoms.
print("\nMulliken Atomic Charges:")
charges = mf.mulliken_pop()[1]
for i, q in enumerate(charges):
    symbol = mol.atom_symbol(i)
    print(f"   {symbol:2} atom {i+1}: Charge = {q:+.3f}")

# Orbital Energies - Useful for visualizing energy levels or building orbital diagrams.
print("\nFirst few Molecular Orbital Energies (Hartree):")
for i, e in enumerate(mo_energy[:10]):
    occ = "occ" if i <= homo_index else "vir"
    print(f"   Orbital {i:2d} ({occ}): Energy = {e:.4f} Hartree")
#This is how we can get forces from this calculation
force = mf.nuc_grad_method().kernel()

In [ ]:
##### Lets visualize the Molecular Orbitals that obtained
from pyscf.tools import cubegen
import py3Dmol
import tempfile
import os

# Choose which orbital to visualize
orbital_index = mol.nelectron // 2 - 1  # <---- HOMO
# To view other orbitals, just change orbital_index to any valid index we outputed in the last cell

# To produce a visualization, we will generate a cube file for the selected orbital
tmp_dir = tempfile.mkdtemp()
cube_filename = os.path.join(tmp_dir, f"mo_{orbital_index}.cube")
cubegen.orbital(mol, cube_filename, mf.mo_coeff[:, orbital_index], nx=40)
# Check the use of cubegen in PySCF https://pyscf.org/pyscf_api_docs/pyscf.tools.html#pyscf.tools.cubegen.orbital

# Reads the cube file
with open(cube_filename) as f:
    cube_data = f.read()

# builds an interactive 3D viewer
viewer = py3Dmol.view(width=400, height=400)
viewer.addModel(cube_data, 'cube')  # Load cube file
viewer.setStyle({'stick':{'radius': 0.1},'sphere':{'radius': 0.3}})      # Show atoms and bonds as sticks
viewer.addVolumetricData(cube_data, 'cube', {'isoval': 0.05, 'color': 'blue', 'opacity': 0.7})
viewer.addVolumetricData(cube_data, 'cube', {'isoval': -0.05, 'color': 'red', 'opacity': 0.7})
viewer.zoomTo()
viewer.show()

# Clean up temp files (optional, but important to know how to do, cause cube files can be heavy)
# os.remove(cube_filename)
# os.rmdir(tmp_dir)

# Question: Those orbitals have the shape you would expect?
# https://chem.libretexts.org/Bookshelves/General_Chemistry/General_Chemistry_Supplement_%28Eames%29/Molecular_Orbital_Theory/MO_Diagrams_for_Water_and_Nitrate_Ion

### What can you do?

Now, you can try to compute a particular system that you like. Maybe try to run something that has charges and different spin states. Maybe try to define a calculation with another functional? Or a small coupled cluster one? Check https://pyscf.org/quickstart.html or https://pyscf.org/user.html for more informations on how to do it and all the posibilities with PySCF.

Be aware that maybe you will need to play first with low values of mf.conv_tol and mf.max_cycle, to be able to do it in the tutorial time. **Be free to skip this task and finish the notebook before.**  

In [ ]:
from pyscf import dft,gto,scf

#mol = gto.M(
#    atom = '''
#    XX  0.000   0.000   0.000''',
#    basis = '...'
#    )

#mf = ...

#mf.kernel()

#mf.analyze()

# ...

## How to use PySCF for Bulk Systems

While it is primarily designed for molecular quantum chemistry using Gaussian basis sets, PySCF also includes support for periodic (bulk) systems through its `pbc` modules. Unlike plane-wave-based codes (e.g., VASP or Quantum ESPRESSO), PySCF handles periodic calculations using Gaussian basis sets with k-point sampling. This makes it well-suited for prototyping periodic electronic structure methods, but it comes with limitations in terms of scalability and performance for large or complex solids. In this section, we illustrate how to define simple periodic systems (by no particular reason, a [diamond](https://diamond-diadem.github.io/) crystal), perform DFT calculations, generate the band structure, and understand what kinds of bulk simulations you can do with PySCF.

In [ ]:
import pyscf.pbc.tools.pyscf_ase as pyscf_ase
import pyscf.pbc.gto as pbcgto
import pyscf.pbc.dft as pbcdft

import matplotlib.pyplot as plt

from ase.build import bulk


In [ ]:
##### Example of setting up a periodic calculation with PySCF using ASE

from ase.build import bulk
from pyscf.pbc import gto as pbcgto
from pyscf.pbc.tools import pyscf_ase

#Builds a diamond structure of carbon using ASE
c = bulk('C', 'diamond', a=3.5668)       # Cubic diamond lattice with lattice constant a = 3.5668 Å
print("Volume of the simulation cell: ",c.get_volume(), "Å³") # Prints the volume of the unit cell, just to check...
#QUESTION: We have the correct density? Each carbon atom has mass of 12 amu (1.9945×10^-23 g).

#Defines a periodic PySCF cell object using ASE atoms
cell = pbcgto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(c)  # Converts ASE atoms to PySCF format
cell.a = c.cell                              # Copies the lattice vectors from the ASE object

cell.basis = 'gth-szv'                   # Basis set: single-zeta valence with Gaussian-type orbitals
cell.pseudo = 'gth-pade'                # Pseudopotential: Goedecker-Teter-Hutter with Pade exchange
#This combination is too simple... Lets use it here just to get some results fast...
cell.verbose = 0                        # Controls the amount of printed output (0 = silent, try changing it and defining a output file)

cell = cell.build(None, None)           # Finalizes and builds the periodic cell object


In [ ]:
##### Generate and visualize a supercell using ASE + py3Dmol
import io
from ase.build import make_supercell, find_optimal_cell_shape

# Finds an optimal 3D integer matrix P such that the supercell has ~256 atoms (just because it makes a nice cube!)
P = find_optimal_cell_shape(c.cell, 256, 'sc')  # 'sc' = simple cubic-like supercell
supercell = make_supercell(c, P)                # Builds the supercell using the transformation matrix P

# Converts the ASE Atoms object to an XYZ format string (in-memory) for visualization
xyz_string = io.StringIO()
supercell.write(xyz_string, format='xyz')       # Writes atomic coordinates in XYZ format to the string buffer

# Creates an interactive 3D viewer with py3Dmol
view = py3Dmol.view(width=600, height=400)

# Loads the atomic structure from the XYZ string into the viewer
view.addModel(xyz_string.getvalue(), 'xyz')

# Sets visualization style: sticks and spheres (ball-and-stick)
view.setStyle({'stick': {'radius': 0.2}, 'sphere': {'radius': 0.5}})

# Automatically zooms to fit the whole structure in the view
view.zoomTo()

# Displays the 3D visualization
view.show()


In [ ]:
##### Define a high-symmetry k-point path for band structure calculation

from ase.dft.kpoints import sc_special_points as special_points, get_bandpath

# Retrieves high-symmetry points for the face-centered cubic (FCC) Brillouin zone
points = special_points['fcc']
G = points['G']     # Gamma point (center of Brillouin zone)
X = points['X']
W = points['W']
K = points['K']
L = points['L']

# Defines a k-point path through high-symmetry points in the Brillouin zone
# Path: L → Γ → X → W → K → Γ
path = get_bandpath([L, G, X, W, K, G], c.cell, npoints=50)  # 50 points between each segment

# Converts the relative k-points to absolute Cartesian coordinates (Bohr⁻¹)
band_kpts = cell.get_abs_kpts(path.kpts)

# Generates x-axis values and labels for plotting (e.g., Γ-X-W...)
x, X, sp_points = path.get_linear_kpoint_axis()


In [ ]:
##### Perform a DFT single-point energy calculation and compute band structure

import time
from pyscf.pbc import dft as pbcdft

# Start timer for SCF calculation
t0 = time.time()

# Define and run a periodic DFT calculation (RKS = spin-restricted Kohn-Sham)
mf = pbcdft.RKS(cell)
print(mf.kernel())                    # Runs the SCF and prints the total energy

# Print timing for SCF step
tf = time.time()
print(f"Time on the Single Point calculation: {tf - t0:.2f} seconds")

# Start timer for band structure calculation
t0 = time.time()

# Computes the band energies at the specified k-points along the high-symmetry path
e_kn = mf.get_bands(band_kpts)[0]     # e_kn is a list of arrays: one band structure per k-point

# Normalize band energies so that the valence band maximum (VBM) is set to 0 eV
vbmax = -99
for en in e_kn:
    vb_k = en[cell.nelectron // 2 - 1]  # Energy of the highest occupied band at each k-point
    if vb_k > vbmax:
        vbmax = vb_k
e_kn = [en - vbmax for en in e_kn]     # Shift all bands by the VBM

# Print timing for band structure step
tf = time.time()
print(f"Time on the Band Structure calculation: {tf - t0:.2f} seconds")


In [ ]:
##### Band structure using a 2×2×2 k-point grid for SCF sampling

import time
from pyscf.pbc import dft as pbcdft

t0 = time.time()
kmf = pbcdft.KRKS(cell, cell.make_kpts([2, 2, 2]))  # <--- 2×2×2 k-point grid. The only thing changing!!!
print(kmf.kernel())
tf = time.time()
print(f"Time on the Single Point calculation: {tf - t0:.2f} seconds")

t0 = time.time()
e_kn_2 = kmf.get_bands(band_kpts)[0]
vbmax = -99
for en in e_kn_2:
    vb_k = en[cell.nelectron // 2 - 1]
    if vb_k > vbmax:
        vbmax = vb_k
e_kn_2 = [en - vbmax for en in e_kn_2]
tf = time.time()
print(f"Time on the Band Structure calculation: {tf - t0:.2f} seconds")


In [ ]:
##### Plot band structures computed from Γ-only and 2×2×2 k-point sampled SCF calculations

import matplotlib.pyplot as plt

au2ev = 27.21139 # Conversion factor: Hartree to electron volts
emin = -1 * au2ev # Energy range for the y-axis (in eV)
emax =  1 * au2ev

plt.figure(figsize=(5, 6))
nbands = cell.nao_nr()  # Number of molecular orbitals (bands) to plot

# Plot first band (n = 0) for both calculations, we are doing it first to set the plot with labels
plt.plot(x, [e[0] * au2ev for e in e_kn],   color='b', label='K = [1,1,1]')   # Γ-point only SCF
plt.plot(x, [e[0] * au2ev for e in e_kn_2], color='r', label='K = [2,2,2]')   # 2×2×2 SCF
# Plot remaining bands (without labels)
for n in range(1, nbands):
    plt.plot(x, [e[n] * au2ev for e in e_kn],   color='b')   # Γ-only bands
    plt.plot(x, [e[n] * au2ev for e in e_kn_2], color='r')   # 2×2×2 bands

# Set x-axis tick labels with symbols for symmetry points
plt.xticks(X, ['$%s$' % n for n in ['L', r'\Gamma', 'X', 'W', 'K', r'\Gamma']])
# Draw vertical lines at each high-symmetry k-point
for p in X:
    plt.plot([p, p], [emin, emax], 'k-')

# Draw a horizontal dotted line at 0 eV (valence band maximum)
plt.plot([0, X[-1]], [0, 0], 'k--')

plt.axis(xmin=0, xmax=X[-1], ymin=emin, ymax=emax)
plt.xlabel('k-vector')
plt.ylabel('Energy (eV)')
plt.legend()
plt.grid(True)
plt.show()